In [1]:
# Imports

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# tensorflow version

tf.__version__

'2.0.0'

In [3]:
# Load dataset and split it into train and test

(_X_train, _y_train), (_X_test, _y_test) = boston_housing.load_data()

In [4]:
print("Trainig_Set Dims: {}\nTest_Set Dims: {}\n".format(_X_train.shape, _X_test.shape))

Trainig_Set Dims: (404, 13)
Test_Set Dims: (102, 13)



In [5]:
# Reshaping _ys to apply Scikit StandardScaler

y_train = _y_train.reshape((-1, 1)).astype('float32')
y_test = _y_test.reshape((-1, 1)).astype('float32')

In [6]:
#Scaling Inputs

def scalerFunc(data):
    return [StandardScaler().fit_transform(dSet).astype('float32') for dSet in data]

X_train, X_test  = scalerFunc((_X_train, _X_test))

In [7]:
# Batch, Learning_rate & epochs definitions

batch = 64
learning_rate = 0.01
epochs = 1000

In [8]:
# A simple Linear Regressor I class. It Inherits from Model Keras Class 

class LinearRegresionI(Model):
    
    def __init__(self, num_predictors):
        super(LinearRegresionI, self).__init__()
        self.w = tf.Variable(tf.random.normal([num_predictors, 1], dtype='float32'),
                                   name='W', trainable=True)
        self.b = tf.Variable(tf.zeros([1, 1], dtype='float32'), name='b', trainable=True)
    
    def call(self, inputs):
        return tf.add(tf.matmul(inputs, self.w), self.b)

In [9]:
# LinearRegressionII instance class
lrI = LinearRegresionI(num_predictors=X_train.shape[1])

In [10]:
# Functions that returns train and test tf.data.Dataset objects 

def dataBatch(dataTrain, dataTest, batch_size=batch):
     return tf.data.Dataset.from_tensor_slices(dataTrain).shuffle(len(dataTrain[0])).batch(batch_size), \
            tf.data.Dataset.from_tensor_slices(dataTest).batch(batch_size)

train_batch, test_batch = dataBatch((X_train, y_train), (X_test, y_test))

In [11]:
# loss function, optimizer and metrics objects definitions

loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_rmse = tf.keras.metrics.RootMeanSquaredError(name='train_rmse')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_rmse = tf.keras.metrics.RootMeanSquaredError(name='test_rmse')

In [12]:
# Learning function. @tf.function, tf.GradientTape

@tf.function
def trainning_test_step(model, loss_fn, optimizer, inputs, labels, Train=True):
    if Train:
        with tf.GradientTape() as tape:
            predictions = model(inputs)
            loss_ = loss_fn(labels, predictions)
            grads = tape.gradient(loss_, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        train_loss(loss_)
        train_rmse(labels, predictions)
        
    predictions = model(inputs)
    loss_ = loss_fn(labels, predictions)
    test_loss(loss_)
    test_rmse(labels, predictions)

In [13]:
# Training Loop

Train_Loss = []
Train_RSME = []

for epoch in range(epochs):
    for train_input, train_label in train_batch:
        trainning_test_step(lrI, loss, optimizer, train_input, train_label)
        Train_Loss.append(train_loss.result().numpy())
        Train_RSME.append(train_rmse.result().numpy())

print("Final Iteration: Epoch: {} Train_Loss {} Train_rmse {}".format(epoch, 
                                                                      Train_Loss[-1], 
                                                                      Train_RSME[-1]))

Final Iteration: Epoch: 999 Train_Loss 24.264299392700195 Train_rmse 4.927485466003418


In [14]:
# Test loop

Test_Loss = []
Test_RSME = []

for epoch in range(epochs):
    for test_input, test_label in test_batch:
        trainning_test_step(lrI, loss, optimizer, test_input, test_label)
        Test_Loss.append(test_loss.result().numpy())
        Test_RSME.append(test_rmse.result().numpy())

print("Final Iteration: Epoch: {} Train_Loss {} Train_rmse {}".format(epoch, 
                                                                      Test_Loss[-1], 
                                                                      Test_RSME[-1]))

Final Iteration: Epoch: 999 Train_Loss 22.139591217041016 Train_rmse 4.717100143432617
